In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6/temp'):
    for filename in filenames:
        a = 1
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os

lst = os.listdir(r'C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6/temp')

In [3]:
len(lst)

10000

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,LSTM, Reshape, SimpleRNN
from keras.layers.recurrent import GRU

Using TensorFlow backend.


In [6]:
import os
import string
import cv2

In [7]:
characters = string.ascii_lowercase+"0123456789"+string.ascii_uppercase
characters

'abcdefghijklmnopqrstuvwxyz0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [8]:
len(characters)

62

In [9]:
all_images =  os.listdir(r'C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6\temp')

In [10]:
all_images[-1], len(all_images)

('01264j_fb013834-b39d-4788-875b-4d069eb3a264.png', 10000)

In [11]:
def preprocess_data():
    
    inp_x = np.zeros((len(all_images), 100,160,1))
    inp_y = np.zeros((len(all_images), 6,len(characters)))
    for idx, image in enumerate(all_images):
        # ----------------prepare input container
        
        
        img = cv2.imread(r"C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6/temp/"+image, cv2.IMREAD_GRAYSCALE)
        #print(np.max(img))
        #plt.imshow(img)
        #print(img.shape)
        
        # ---------------------Scale images --------------
        
        img = img/255.0
        
        image_txt = image[0:6]
        target_oht = np.zeros((5,len(characters)))
        if(len(image_txt)<7):
            img = np.reshape(img, (100,160,1))
            inp_x[idx] = img
            
             # ------------------Define targets and code them using OneHotEncoding
            target_oht = np.zeros((6,len(characters)))
            for k, char in enumerate(image_txt):
                target_oht[k, characters.find(char)] = 1
            inp_y[idx] = target_oht
    return inp_x, inp_y
        
               
        
        
        

In [12]:
X, Y = preprocess_data()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_x ,test_x, train_y ,test_y = train_test_split(X,Y, shuffle = True, test_size = .1)

In [15]:
train_x.shape,train_y.shape, test_x.shape,test_y.shape

((9000, 100, 160, 1), (9000, 6, 62), (1000, 100, 160, 1), (1000, 6, 62))

In [16]:
model = Sequential()

In [17]:
def prepare_model():
    model.add(Conv2D(128, (3,3), input_shape = (100,160,1) ,padding = 'same', activation = 'relu'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same',))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size =(3,3),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,3),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same'))
  
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,3),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (1,1),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (1,1),padding ='same'))
    print(model.output_shape)
    model.add(Reshape((model.output_shape[1], model.output_shape[2]*model.output_shape[3])))
    print(model.output_shape)
    #model.add(LSTM(50, batch_input_shape=(2,5,36),stateful = True, return_sequences = True))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
  


    model.add(Dense(62, activation = "softmax"))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
   
    return model
    

In [18]:
model =prepare_model()

(None, 6, 9, 32)
(None, 6, 288)


In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 160, 128)     1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 80, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 80, 64)        73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 80, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 27, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 17, 27, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 9, 32)         

In [20]:
#model = create_model()
hist = model.fit(train_x, train_y, batch_size=32, epochs=50,verbose=1, validation_data=(test_x, test_y))

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 115s 13ms/step - loss: 1.7059 - accuracy: 0.5455 - val_loss: 1.5644 - val_accuracy: 0.5612
Epoch 2/50
9000/9000 [==============================] - 98s 11ms/step - loss: 1.5656 - accuracy: 0.5605 - val_loss: 1.5619 - val_accuracy: 0.5610
Epoch 3/50
9000/9000 [==============================] - 94s 10ms/step - loss: 1.5611 - accuracy: 0.5609 - val_loss: 1.5561 - val_accuracy: 0.5610
Epoch 4/50
9000/9000 [==============================] - 95s 11ms/step - loss: 1.5564 - accuracy: 0.5601 - val_loss: 1.5570 - val_accuracy: 0.5600
Epoch 5/50
9000/9000 [==============================] - 97s 11ms/step - loss: 1.5541 - accuracy: 0.5596 - val_loss: 1.5525 - val_accuracy: 0.5605
Epoch 6/50
9000/9000 [==============================] - 96s 11ms/step - loss: 1.5521 - accuracy: 0.5606 - val_loss: 1.5511 - val_accuracy: 0.5655
Epoch 7/50
9000/9000 [==============================] - 95s 11ms/step - los

In [24]:
score= model.evaluate(test_x,test_y,verbose=1)
print('Test Loss and accuracy:', score)

1000/1000 [==============================] - 2s 2ms/step
Test Loss and accuracy: [0.5314826798439026, 0.8654999732971191]


In [25]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print("img",img.shape)
    if(img is not None):
        img = cv2.resize(img, (160,100))
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (6, 62))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += characters[l]
    return capt#, sum(probs) / 5

In [27]:
for i in all_images[:20]:
    #print(i)
    #model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
    print(i[:6],"predicted : ",predict(r'C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6/temp/'+i))
    

012345 predicted :  01234s
012346 predicted :  0123g6
012347 predicted :  012347
012348 predicted :  0123A8
012349 predicted :  012349
01234A predicted :  01234A
01234a predicted :  01234a
01234B predicted :  01234B
01234b predicted :  01234b
01234c predicted :  01234c
01234C predicted :  01234C
01234d predicted :  0123Ad
01234D predicted :  01234O
01234e predicted :  01234e
01234E predicted :  01234E
01234F predicted :  01234F
01234f predicted :  01234f
01234G predicted :  01234G
01234g predicted :  01234g
01234H predicted :  01234H


In [28]:
# --------------------Saving Model --------------------------------------


model.save_weights(r'C:\Users\mpspa\Desktop\Github\MLDLNLP\captcha_destroyer\model_files/model_weights.h5')

# Save the model architecture
with open(r'C:\Users\mpspa\Desktop\Github\MLDLNLP\captcha_destroyer\model_files/model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [30]:
# ------------------------------Loading ---------------------------------

from keras.models import model_from_json

# Model reconstruction from JSON file
with open(r'C:\Users\mpspa\Desktop\Github\MLDLNLP\captcha_destroyer\model_files\model_architecture.json', 'r') as f:
    model1 = model_from_json(f.read())

# Load weights into the new model
model1.load_weights(r'C:\Users\mpspa\Desktop\Github\MLDLNLP\captcha_destroyer\model_files\model_weights.h5')

In [31]:
# Define function to predict captcha
def predict1(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print("img",img.shape)
    if(img is not None):
        img = cv2.resize(img, (160,100))
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model1.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (6, 62))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += characters[l]
    return capt#, sum(probs) / 5

In [32]:
for i in all_images[:20]:
    #print(i)
    #model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
    print(i[:6],"predicted : ",predict1(r'C:\Users\mpspa\Desktop\Kaggle\Captcha\captcha-tensorflow-master\datasets\images\char-6-epoch-6/temp/'+i))
    

012345 predicted :  01234s
012346 predicted :  0123g6
012347 predicted :  012347
012348 predicted :  0123A8
012349 predicted :  012349
01234A predicted :  01234A
01234a predicted :  01234a
01234B predicted :  01234B
01234b predicted :  01234b
01234c predicted :  01234c
01234C predicted :  01234C
01234d predicted :  0123Ad
01234D predicted :  01234O
01234e predicted :  01234e
01234E predicted :  01234E
01234F predicted :  01234F
01234f predicted :  01234f
01234G predicted :  01234G
01234g predicted :  01234g
01234H predicted :  01234H
